In [16]:
import sys
import re
import pymongo
import json
import time
import datetime
import requests
from bs4 import BeautifulSoup

dbname = "fdac18mp2" #please use this database
collname = "glprj_bbass11" #please modify so you store data in your collection
# beginning page index
begin = "0"
client = pymongo.MongoClient()
letter = "b"

db = client[dbname]
coll = db[collname]


beginurl = "https://gitlab.com/api/v4/projects?archived=false&membership=false&order_by=created_at&owned=false&page=" + begin + \
    "&per_page=99&simple=false&sort=desc&starred=false&statistics=false&with_custom_attributes=false&with_issues_enabled=false&with_merge_requests_enabled=false"


gleft = 10

header = {'per_page': 99}

# check remaining query chances for rate-limit restriction
def wait(left):
    global header
    while (left < 20):
        l = requests.get('https://gitlab.com/api/v4/projects', headers=header)
        if (l.ok):
            left = int(l.headers.get('RateLimit-Remaining'))
        time .sleep(60)
    return left

# send queries and extract urls 
def get(url, coll):

    global gleft
    global header
    global bginnum
    gleft = wait(gleft)
    values = []
    size = 0
    count = 0

    try:
        r = requests .get(url, headers=header)
        time .sleep(0.5)
        # got blocked
        if r.status_code == 403:
            return "got blocked", str(bginnum)
        if (r.ok):

            gleft = int(r.headers.get('RateLimit-Remaining'))
            lll = r.headers.get('Link')
            t = r.text
            array = json.loads(t)
            
            for el in array:
                if el['name'].lower().startswith(letter):
                    el['site'] = "git"
                    count += 1
                    coll.insert_one(el)
                    if count > 49:
                        return
                
 
            #next page
            while ('; rel="next"' in lll):
                gleft = int(r.headers.get('RateLimit-Remaining'))
                gleft = wait(gleft)
                # extract next page url
                ll = lll.replace(';', ',').split(',')
                url = ll[ll.index(' rel="next"') -
                         1].replace('<', '').replace('>', '').lstrip()
             
                try:
                    r = requests .get(url, headers=header)
                    if r.status_code == 403:
                        return "got blocked", str(bginnum)
                    if (r.ok):
                        lll = r.headers.get('Link')
                        t = r.text
                        array1 = json.loads(t)
                        for el in array1:
                            if el['name'].lower().startswith(letter):
                                el['site'] = "git"
                                count += 1
                                coll.insert_one(el)
                                if count > 49:
                                    return
                    else:
                        sys.stderr.write("url can not found:\n" + url + '\n')
                        return 
                except requests.exceptions.ConnectionError:
                    sys.stderr.write('could not get ' + url + '\n')

        else:
            sys.stderr.write("url can not found:\n" + url + '\n')
            return

    except requests.exceptions.ConnectionError:
        sys.stderr.write('could not get ' + url + '\n')
    except Exception as e:
        sys.stderr.write(url + ';' + str(e) + '\n')

In [3]:
#start retrieving        
get(beginurl,coll)

In [57]:
proj_array = np.array([])


for proj in coll.find({}):
    if proj['name'].lower().startswith(letter):
        projs = (proj)
        proj_array = np.append(proj_array, projs)
        
print(proj_array[0])

{'namespace': {'full_path': 'kchoppin', 'name': 'kchoppin', 'path': 'kchoppin', 'kind': 'user', 'parent_id': None, 'id': 3532394}, 'created_at': '2018-10-10T19:35:15.074Z', 'web_url': 'https://gitlab.com/kchoppin/birthday-bitcoin', 'readme_url': None, 'path': 'birthday-bitcoin', 'default_branch': None, 'description': '', 'id': 8801814, 'ssh_url_to_repo': 'git@gitlab.com:kchoppin/birthday-bitcoin.git', 'path_with_namespace': 'kchoppin/birthday-bitcoin', 'last_activity_at': '2018-10-10T19:35:15.074Z', 'star_count': 0, 'http_url_to_repo': 'https://gitlab.com/kchoppin/birthday-bitcoin.git', 'name': 'Birthday Bitcoin', '_id': ObjectId('5bbe54dff318b932af8a62c5'), 'avatar_url': None, 'name_with_namespace': 'Kevin Choppin / Birthday Bitcoin', 'tag_list': [], 'forks_count': 0}


In [32]:
# Define number of search pages to iterate
Pages = 50

url = []

# Collect URLs
for i in range(1,Pages):
    url.append("https://sourceforge.net/directory/os%3Awindows/?q=b&page=" + str(i))
    
# Get URL responses
response= [requests.get(link) for link in url]

In [33]:
# Function to select only unique entries in list
def unique(seq): 
   checked = []
   for e in seq:
       if e not in checked:
           checked.append(e)
   return checked

In [42]:
# Iterate through responses and extract project names
b_proj=[]
for resp in response:
    # Parse web response
    html_soup= BeautifulSoup(resp.text, 'html.parser')
    soup_string= str(html_soup)
    
    # Match all projects
    regex = "(?<=\/projects\/).+?(?=\/)"
    matches = re.findall(regex, soup_string)
    b_matches= [i for i in matches if i.startswith('b')]
    b_proj.append(b_matches)

In [43]:
print(b_proj)

[['b-e-c', 'b-e-c', 'b-e-c', 'b-e-c', 'b-e-c', 'bmaxwell', 'bmaxwell', 'bmaxwell', 'bmaxwell', 'bmaxwell', 'b-i-b-l-e', 'b-i-b-l-e', 'b-i-b-l-e', 'b-i-b-l-e', 'b-gaming-for-tester', 'b-gaming-for-tester', 'b-gaming-for-tester', 'b-gaming-for-tester', 'bowtie-bio', 'bowtie-bio', 'bowtie-bio', 'bowtie-bio', 'bowtie-bio'], ['basszxtune', 'basszxtune', 'basszxtune', 'basszxtune'], ['b-basic', 'b-basic', 'b-basic', 'b-basic', 'bincmp', 'bincmp', 'bincmp', 'bincmp'], ['btreeindextodbf', 'btreeindextodbf', 'btreeindextodbf', 'btreeindextodbf'], ['bcpl', 'bcpl', 'bcpl', 'bcpl'], ['b-ulysse-cc', 'b-ulysse-cc', 'b-ulysse-cc', 'b-ulysse-cc', 'benchmarksql2', 'benchmarksql2', 'benchmarksql2', 'benchmarksql2', 'bossa-fork', 'bossa-fork', 'bossa-fork', 'bossa-fork'], ['bliteos', 'bliteos', 'bliteos', 'bliteos', 'bgaming', 'bgaming', 'bgaming', 'bgaming', 'bcomp', 'bcomp', 'bcomp', 'bcomp'], ['bartrefiner', 'bartrefiner', 'bartrefiner', 'bartrefiner', 'b-linux', 'b-linux', 'b-linux', 'b-linux', 'blac

In [45]:
# Make list of unique project names
b_list = unique([item for sublist in b_proj for item in sublist])

print(len(b_list))

88


In [49]:
# Iterate through projects, check status and save link
sf_api= "https://sourceforge.net/rest/p/"

projects = []
proj = []
for proj in b_list:
    resp= requests.get(sf_api+proj)
    if(resp.status_code == 404):
        continue
    text= json.loads(resp.text)
    if(text['status']=='active'):
        projects.append(text)

In [52]:
print(projects[0])

{'preferred_support_url': '', 'labels': ['files', 'file', 'extension'], 'shortname': 'b-e-c', 'video_url': None, 'icon_url': 'https://sourceforge.net/p/b-e-c/icon', 'categories': {'os': [{'fullname': 'Win2K', 'id': 420, 'shortname': 'mswin_2000', 'fullpath': 'Operating System :: Modern (Vendor-Supported) Desktop Operating Systems :: Win2K'}, {'fullname': 'WinXP', 'id': 419, 'shortname': 'mswin_xp', 'fullpath': 'Operating System :: Modern (Vendor-Supported) Desktop Operating Systems :: WinXP'}, {'fullname': 'All 32-bit MS Windows (95/98/NT/2000/XP)', 'id': 435, 'shortname': 'mswin_all32bit', 'fullpath': 'Operating System :: Grouping and Descriptive Categories :: All 32-bit MS Windows (95/98/NT/2000/XP)'}, {'fullname': '64-bit MS Windows', 'id': 655, 'shortname': 'win64', 'fullpath': 'Operating System :: Grouping and Descriptive Categories :: 64-bit MS Windows'}, {'fullname': 'Vista', 'id': 657, 'shortname': 'vista', 'fullpath': 'Operating System :: Modern (Vendor-Supported) Desktop Oper

In [53]:
for el in projects:
        el['site'] = "SF"
        coll.insert_one(el)

In [58]:
proj_array = np.array([])


for proj in coll.find({}):
    if proj['name'].lower().startswith(letter):
        projs = (proj)
        proj_array = np.append(proj_array, projs)
        
print(len(proj_array))

3024


In [59]:
print(proj_array[3000])

{'icon_url': None, 'screenshots': [], 'site': 'SF', 'short_description': 'B-Client is a Windows DLL that provides internet communications protocols.  This C++ DLL supports SMTP, POP, IMAP and NNTP protocols with support for other open internet communication protocols in the future.', 'name': 'B-Client Communications Protocol Library', '_id': '5163065934309d2f05b1a601', 'tools': [{'tool_label': 'Summary', 'url': '/p/bclient/summary/', 'name': 'summary', 'mount_point': 'summary', 'installable': False, 'sourceforge_group_id': 126638, 'mount_label': 'Summary', 'icons': {'32': 'images/sftheme/32x32/blog_32.png', '48': 'images/sftheme/48x48/blog_48.png', '24': 'images/sftheme/24x24/blog_24.png'}}, {'tool_label': 'Support', 'url': '/p/bclient/support/', 'name': 'support', 'mount_point': 'support', 'installable': False, 'mount_label': 'Support', 'icons': {'32': 'images/sftheme/32x32/blog_32.png', '48': 'images/sftheme/48x48/blog_48.png', '24': 'images/sftheme/24x24/blog_24.png'}}, {'tool_label